## Setup 

In this document we make sure all libraries can load, and we download the inital relquired data-sets. etc.
As a reminder we tried to install a host of libraries. If those functions did not work, the follow codebox will not execute.

**As a reminder again the install instructions:** 

Install NumPy: ```conda install -c conda-forge numpy ```  
Install MatplotLib: ```conda install -c conda-forge matplotlib ```  
Install from: https://pypi.org/project/numpy_ringbuffer/ ```pip install numpy_ringbuffer```  
Install pyTorch: https://pytorch.org/get-started/locally/ so e.g. ```conda install pytorch torchvision -c pytorch```  
Install opencv2: ```conda install -c conda-forge  opencv=3.4.1```  
Install sklearn: ```conda install scikit-learn```  
Install librosa: ```conda install -c conda-forge librosa ```  
Install PyAudio: ```conda install -c anaconda pyaudio```  

In [1]:
import csv
import os
import wave
import time
import multiprocessing
from threading import Thread
import os
print("Found all build-in libraries.")
import numpy as np
from numpy_ringbuffer import RingBuffer
import matplotlib.pyplot as plt
print("Found all math libraries.")
import pyaudio
import librosa
print("Found all audio libraries.")
import torch
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix
print("Found all machine learning libraries.")
import cv2 as cv
print("Found the computer vision library.")


Found all build-in libraries.
Found all math libraries.
Found all audio libraries.
Found all machine learning libraries.
Found the computer vision librarie.


If the above code runs without a problem, all libraries have installed and can be loaded. While it does not gurantee that everything will work, this is a good sign.

## Getting data, lots of data

One of the iomportant aspects for neural networks is that you need a lot of data that is diverse enough toreally teach the network the difference between the classes sounds that you are trting to seperate.

For the first exsersize and run down, we will use a typical mashin learning example, Cats vs. Dogs, but this time in audio form.Go head on over to the following kaggle link, make an account and download the data: Download: [www.kaggle.com/mmoreaux/audio-cats-and-dogs](https://www.kaggle.com/mmoreaux/audio-cats-and-dogs/downloads/audio-cats-and-dogs.zip/). After it finished downloading, unzip the folder and move it to to the  ``AudioData/`` folder in the repository.

---

In the meantime, whiloe things are downloading, and you are working go ahead and stat downloading the Urban sound data sets [here on kaggle](https://www.kaggle.com/pavansanagapati/urban-sound-classification). When its finished, again unzip and move it to the ``AudioData/`` folder.

---

While the data is downloading lease have alook at the document []() where we explain the data flow for this project.


## Transforming data-sets
As mentioned befor, We need to move from one folder full of data into a folder for each class. In somoe cases we also need to split the dat wbetween test and train data. Why that is important we will come back to later, but for now understand that its important to have data to train with and data to test against.

The following script will look at the cats-dogs data folder and use the .csv file seperate them into the different folders.

In [ ]:
SOURCEFOLDER= '../AudioData/audio-cats-and-dogs/cats_dogs/'
CSV = '../AudioData/audio-cats-and-dogs/train_test_split.csv'
folderNames=['train','test']  # We define a list of folders for us to store the data in
classes = ['cat','dog'] # Similar like the row above, we define the classes that will turn into folders later
ClassFolderPath={} # Dictionaries are a handy way to keep track of complex data like a folder path

'''
In the following lines of the script, we make sure that the target folders exsist. 
The first for loops checks for the test and train folder and the inside for loop looks for the class folders.
If they do not exsists they are created.
The path are all added to the dictionary to use later.
'''
for i in folderNames: 
    TopLevelFolder = os.path.join(SOURCEFOLDER,i)
    if not os.path.exists(TopLevelFolder):
        os.makedirs(TopLevelFolder)
    TempDict={}
    for c in classes:
        LowLevelFolder = os.path.join(TopLevelFolder,c)
        if not os.path.exists(LowLevelFolder):
            os.makedirs(LowLevelFolder)
        TempDict[c] = LowLevelFolder
    ClassFolderPath[i] = TempDict;

'''

We now have a nested dictionary with the final folder path.
This allows us to find the folder path e.g. 
for the cats - training folder like this ClassFolderPath['train']['cat'].


This makes it very easy to seperate out the different files and move them to the correct folder. 


'''
    
    
    
with open(CSV, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        for key in row.keys():
            if(len(key)==0):
                continue;
            if(len(row[key])==0):
                continue
            outDirectoryPath=''    
            if('cat'in key and 'train' in key):
                outDirectoryPath= ClassFolderPath['train']['cat']
            elif('cat'in key and 'test' in key):
                outDirectoryPath= ClassFolderPath['test']['cat']
            elif('dog'in key and 'train' in key):
                outDirectoryPath= ClassFolderPath['train']['dog']
            elif('dog'in key and 'test' in key):
                outDirectoryPath= ClassFolderPath['test']['dog']
            inFilePath = os.path.join(SOURCEFOLDER,row[key])
            outFilePath =os.path.join(outDirectoryPath,row[key])
            if(os.path.isfile(inFilePath) and not os.path.isfile(outFilePath)):
                os.rename(inFilePath,outFilePath)
                print("Moved file: "+row[key])
            else:
                print("Skipping file: "+row[key]+ "as it already exsists")

**HOTTIP:**
If you write your own code, while testing, instead of calling ``os.rename(inFilePath,outFilePath)`` first print out the path like ``print(inFilePath,outFilePath)`` and compare them by and. Not only can it be a if the code does not work correclty but if you are not carfull and move files into or away from lets say ``/``(root) you can mess up your system really easily. In other words, make sure your code works befor you start moving around big chuncks of files.


## Urban sound preperation

The test files are not usefule to us. We can only use the train data that contain labels. This is why we only reorganize the ``Train`` folder.

In [ ]:
SOURCEFOLDER= '../AudioData/urban-sound-classification/train/Train/'
CSV = '../AudioData/urban-sound-classification/train/train.csv'
with open(CSV, newline='') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row  in readCSV:
        if( row[0]=='ID'):
            continue;   
        targetFolder = os.path.join(SOURCEFOLDER,row[1])
        if not os.path.exists(targetFolder):
            os.makedirs(targetFolder)
        filename = row[0]+".wav"
        inFilePath = os.path.join(SOURCEFOLDER,filename)
        outFilePath = os.path.join(targetFolder,filename)
        if(os.path.isfile(inFilePath) and not os.path.isfile(outFilePath)):
            print("Moved file: "+filename)
            os.rename(inFilePath,outFilePath)
        else:
            print("Skipping file: "+filename+ "as it already exsists")
       

## Appendix

### Handy Links

https://stackoverflow.com/questions/8858008/how-to-move-a-file-in-python    how to move a file in python
